# Import Libraries

In [ ]:
from casadi import *
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.io import loadmat
from scipy.interpolate import PchipInterpolator, interp1d

# Define Test Tire

## 1. Tire stolen from Li Auto

In [ ]:
def tire_Li(Fz, camber, lamda, alpha):
    # Convert angles
    alpha = tan(alpha)
    camber = sin(camber)

    Fz0 = 4607  # N
    R0 = 0.357  # m
    dfz = (Fz - Fz0) / Fz0

    ## === Pure Longitudinal Force Fx0 ===
    pcx1 = 1.65
    pdx1, pdx2, pdx3 = 1.3, -0.15, 0
    pex1, pex2, pex3, pex4 = 0, 0, 0, 0
    pkx1, pkx2, pkx3 = 20, 0, 0
    pvx1, pvx2 = 0, 0
    phx1, phx2 = 0, 0

    Cx = pcx1
    Shx = phx1 + phx2 * dfz
    lamda_x = lamda + Shx
    Dx = (pdx1 + pdx2 * dfz) * (1 - pdx3 * camber**2) * Fz
    Ex = (pex1 + pex2 * dfz + pex3 * dfz**2) * (1 - pex4 * sign(lamda_x))
    Kx = Fz * (pkx1 + pkx2 * dfz) * exp(pkx3 * dfz)
    Bx = Kx / (Cx * Dx + 1e-6)
    Svx = Fz * (pvx1 + pvx2 * dfz)
    Fx0 = Dx * sin(Cx * atan(Bx * lamda_x - Ex * (Bx * lamda_x - atan(Bx * lamda_x)))) + Svx

    ## === Combined Fx ===
    rbx1, rbx2, rcx1 = 20, 15, 1
    rex1, rex2, rhx1 = 0, 0, 0
    Bxa = rbx1 * cos(atan(rbx2 * lamda))
    Cxa = rcx1
    Exa = rex1 + rex2 * dfz
    Shxa = rhx1
    alpha_s = alpha + Shxa
    Gxa = cos(Cxa * atan(Bxa * alpha_s - Exa * (Bxa * alpha_s - atan(Bxa * alpha_s)))) / \
          cos(Cxa * atan(Bxa * Shxa - Exa * (Bxa * Shxa - atan(Bxa * Shxa))))
    Fx = Fx0 * Gxa

    ## === Pure Lateral Force Fy0 ===
    pcy1 = 1.529245171
    pdy1, pdy2, pdy3 = 1.038581713, -0.179464279, -0.683745879
    pey1, pey2, pey3, pey4 = -0.39012519, -0.611160145, -0.040677215, -14.95422065
    pky1, pky2, pky3 = -26.056726760, 2.106504624, 0.499984439
    pvy1, pvy2, pvy3, pvy4 = 0, 0, 0, 0
    phy1, phy2, phy3 = 0, 0, 0

    Cy = pcy1
    Shy = phy1 + phy2 * dfz + phy3 * camber
    alpha_y = alpha + Shy
    uy = (pdy1 + pdy2 * dfz) * (1 - pdy3 * camber**2)
    Dy = uy * Fz
    Ey = (pey1 + pey2 * dfz) * (1 - (pey3 + pey4 * camber) * sign(alpha_y))
    Ky = pky1 * Fz0 * sin(2 * atan(Fz / (pky2 * Fz0))) * (1 - pky3 * fabs(camber))
    By = Ky / (Cy * Dy + 1e-6)
    Svy = Fz * (pvy1 + pvy2 * dfz + (pvy3 + pvy4 * dfz) * camber)
    Fy0 = Dy * sin(Cy * atan(By * alpha_y - Ey * (By * alpha_y - atan(By * alpha_y)))) + Svy

    ## === Combined Fy ===
    rby1, rby2, rby3 = 10, 10, -0.027856
    rcy1 = 1
    rey1, rey2 = 0, 0
    rhy1, rhy2 = 0, 0
    rvy1, rvy2, rvy3, rvy4, rvy5, rvy6 = 0, 0, 0, 20, 2, 10

    Bya = rby1 * cos(atan(rby2 * (alpha - rby3)))
    Cya = rcy1
    Eya = rey1 + rey2 * dfz
    Shya = rhy1 + rhy2 * dfz
    lamda_s = lamda + Shya
    Gya = cos(Cya * atan(Bya * lamda_s - Eya * (Bya * lamda_s - atan(Bya * lamda_s)))) / \
          cos(Cya * atan(Bya * Shya - Eya * (Bya * Shya - atan(Bya * Shya))))
    Svyr = uy * Fz * (rvy1 + rvy2 * dfz + rvy3 * camber) * cos(atan(rvy4 * alpha)) * sin(rvy5 * atan(rvy6 * lamda))
    Fy = Fy0 * Gya + Svyr

    ## === Aligning Torque Mz ===
    qbz1, qbz2, qbz3, qbz4, qbz5 = 11.61018865, -0.826853524, -3.513951124, 0.063555994, -1.014358836
    qcz1 = 1.064991884
    qdz1, qdz2, qdz3, qdz4 = 0.091227598, -0.001965108, 0.341001609, -3.426482155
    qez1, qez2, qez3, qez4, qez5 = -9.044014009, 10, -19.59192084, 0.156693082, -0.708977656

    Ct = qcz1
    Bt = (qbz1 + qbz2 * dfz + qbz3 * dfz**2) * (1 + qbz4 * camber + qbz5 * fabs(camber))
    Dt = Fz * (qdz1 + qdz2 * dfz) * (1 + qdz3 * camber + qdz4 * camber**2) * R0 / Fz0
    Et = (qez1 + qez2 * dfz + qez3 * dfz**2) * (1 + (qez4 + qez5 * camber) * (2 / np.pi) * atan(Bt * Ct * alpha))
    t = Dt * cos(Ct * atan(Bt * alpha - Et * (Bt * alpha - atan(Bt * alpha)))) * cos(alpha)
    Mz = -t * Fy0

    return Fx0, Fy0, Fx, Fy, Mz


## 2. Hoosier 43075

In [ ]:
# TODO: Refine tire parameters to a simple version
def tire_Hoosier(Fz, camber, lamda, alpha):
    # Convert angles
    alpha = tan(alpha)
    camber = sin(camber)

    Fz0 = 656  # N
    R0 = 0.2032  # m
    dfz = (Fz - Fz0) / Fz0

    ## === Pure Longitudinal Force Fx0 ===
    pcx1=1.5;
    pdx1=1.5; pdx2=-0.544867 ; pdx3=12.5804 ;
    pex1=0; pex2=0.406217 ;pex3=-3.57069; pex4=0.562024;
    pkx1=68.0819; pkx2=-9.24165e-05; pkx3=-0.470674;
    pvx1=0; pvx2=0;
    phx1=0; phx2=0;

    Cx = pcx1
    Shx = phx1 + phx2 * dfz
    lamda_x = lamda + Shx
    Dx = (pdx1 + pdx2 * dfz) * (1 - pdx3 * camber**2) * Fz
    Ex = (pex1 + pex2 * dfz + pex3 * dfz**2) * (1 - pex4 * sign(lamda_x))
    Kx = Fz * (pkx1 + pkx2 * dfz) * exp(pkx3 * dfz)
    Bx = Kx / (Cx * Dx + 1e-6)
    Svx = Fz * (pvx1 + pvx2 * dfz)
    Fx0 = Dx * sin(Cx * atan(Bx * lamda_x - Ex * (Bx * lamda_x - atan(Bx * lamda_x)))) + Svx

    ## === Combined Fx ===
    rbx1=10.7751;rbx2=8.85735;
    rcx1=1.12418;
    rex1=0.0131769;rex2=0.0131769;
    rhx1=0.00590496;
    Bxa = rbx1 * cos(atan(rbx2 * lamda))
    Cxa = rcx1
    Exa = rex1 + rex2 * dfz
    Shxa = rhx1
    alpha_s = alpha + Shxa
    Gxa = cos(Cxa * atan(Bxa * alpha_s - Exa * (Bxa * alpha_s - atan(Bxa * alpha_s)))) / \
          cos(Cxa * atan(Bxa * Shxa - Exa * (Bxa * Shxa - atan(Bxa * Shxa))))
    Fx = Fx0 * Gxa

    ## === Pure Lateral Force Fy0 ===
    pcy1=1.24038 ;
    pdy1=-1.4; pdy2=0.285186; pdy3=3.89743;
    pey1=-0.0348209; pey2=-0.166206;pey3=0.944869; pey4=1.94872;
    pky1=-45.6002; pky2=1.61754; pky3=1.3621;
    pvy1=0; pvy2=-0;pvy3=0;pvy4=0;
    phy1=0 ; phy2=0;phy3=0;

    Cy = pcy1
    Shy = phy1 + phy2 * dfz + phy3 * camber
    alpha_y = alpha + Shy
    uy = (pdy1 + pdy2 * dfz) * (1 - pdy3 * camber**2)
    Dy = uy * Fz
    Ey = (pey1 + pey2 * dfz) * (1 - (pey3 + pey4 * camber) * sign(alpha_y))
    Ky = pky1 * Fz0 * sin(2 * atan(Fz / (pky2 * Fz0))) * (1 - pky3 * fabs(camber))
    By = Ky / (Cy * Dy + 1e-6)
    Svy = Fz * (pvy1 + pvy2 * dfz + (pvy3 + pvy4 * dfz) * camber)
    Fy0 = Dy * sin(Cy * atan(By * alpha_y - Ey * (By * alpha_y - atan(By * alpha_y)))) + Svy

    ## === Combined Fy ===
    rby1=18.1241;rby2=14.0025;rby3=-0.00834201;
    rcy1=0.952494;
    rey1=-0.243877;rey2=0.00680701;
    rhy1=0.00698677;rhy2=0.0126997;
    rvy1=0.00644359;rvy2=0.0573954;rvy3=0.00260529;rvy4=8.85735;rvy5=1.90793;rvy6=-12.8609;

    Bya = rby1 * cos(atan(rby2 * (alpha - rby3)))
    Cya = rcy1
    Eya = rey1 + rey2 * dfz
    Shya = rhy1 + rhy2 * dfz
    lamda_s = lamda + Shya
    Gya = cos(Cya * atan(Bya * lamda_s - Eya * (Bya * lamda_s - atan(Bya * lamda_s)))) / \
          cos(Cya * atan(Bya * Shya - Eya * (Bya * Shya - atan(Bya * Shya))))
    Svyr = uy * Fz * (rvy1 + rvy2 * dfz + rvy3 * camber) * cos(atan(rvy4 * alpha)) * sin(rvy5 * atan(rvy6 * lamda))
    Fy = Fy0 * Gya + Svyr

    ## === Aligning Torque Mz ===
    qbz1=8.19892 ;qbz2=2.73193 ;qbz3=-4.01805 ;qbz4=-0.429117;qbz5=0.877475;
    qcz1=1.41804;
    qdz1=0.195153;qdz2=-0.0453526;qdz3=0.387762;qdz4=-3.95699;
    qez1=-0.236852;qez2=1.5128;qez3=-1.43111;qez4=0.097797;qez5=0.967582;

    Ct = qcz1
    Bt = (qbz1 + qbz2 * dfz + qbz3 * dfz**2) * (1 + qbz4 * camber + qbz5 * fabs(camber))
    Dt = Fz * (qdz1 + qdz2 * dfz) * (1 + qdz3 * camber + qdz4 * camber**2) * R0 / Fz0
    Et = (qez1 + qez2 * dfz + qez3 * dfz**2) * (1 + (qez4 + qez5 * camber) * (2 / np.pi) * atan(Bt * Ct * alpha))
    t = Dt * cos(Ct * atan(Bt * alpha - Et * (Bt * alpha - atan(Bt * alpha)))) * cos(alpha)
    Mz = -t * Fy0

    return Fx0, Fy0, Fx, Fy, Mz

# Define Vehicle Model
## 1. Double Track Vehicle Model

In [ ]:
def build_vehicle_model_opti(Vel, Vehicle, Env, tire_fun):
    opti = Opti()

    # === 1. Variables ===
    delta = opti.variable(); opti.subject_to(delta >= 0*pi/180); opti.subject_to(delta <= 100*pi/180)
    beta = opti.variable(); opti.subject_to(beta >= -20*pi/180); opti.subject_to(beta <= 20*pi/180)
    dpsi = opti.variable(); opti.subject_to(dpsi >= 0); opti.subject_to(dpsi <= pi)
    Sxfl = opti.variable(); opti.subject_to(Sxfl >= -0.5); opti.subject_to(Sxfl <= 0.5)
    Sxfr = opti.variable(); opti.subject_to(Sxfr >= -0.5); opti.subject_to(Sxfr <= 0.5)
    Sxrl = opti.variable(); opti.subject_to(Sxrl >= -0.5); opti.subject_to(Sxrl <= 0.5)
    Sxrr = opti.variable(); opti.subject_to(Sxrr >= -0.5); opti.subject_to(Sxrr <= 0.5)
    ax = opti.variable()
    ay = opti.variable()

    # === 2. Velocities ===
    dx = Vel * cos(beta)
    dy = Vel * sin(beta)

    delta_fl = Vehicle['Strmap_fl'](delta) - Vehicle['Static_Toe_Front']
    delta_fr = Vehicle['Strmap_fr'](delta) + Vehicle['Static_Toe_Front']
    delta_rl = -Vehicle['Static_Toe_Rear']
    delta_rr = Vehicle['Static_Toe_Rear']

    # === 3. Slip Angles ===
    Safl = -delta_fl + atan((dy + Vehicle['lf']*dpsi)/(dx - Vehicle['Bf']/2*dpsi))
    Safr = -delta_fr + atan((dy + Vehicle['lf']*dpsi)/(dx + Vehicle['Bf']/2*dpsi))
    Sarl = -delta_rl + atan((dy - Vehicle['lr']*dpsi)/(dx - Vehicle['Br']/2*dpsi))
    Sarr = -delta_rr + atan((dy - Vehicle['lr']*dpsi)/(dx + Vehicle['Br']/2*dpsi))

    # === 4. Aero ===
    F_Liftf = 0.5 * Vehicle['ClA'] * Vehicle['rho'] * dx**2 * Vehicle['CoP']
    F_Liftr = 0.5 * Vehicle['ClA'] * Vehicle['rho'] * dx**2 * (1 - Vehicle['CoP'])
    F_drag = 0.5 * Vehicle['CdA'] * Vehicle['rho'] * dx**2

    # === 5. Normal Loads ===
    den = Vehicle['K_phif'] + Vehicle['K_phir'] - Vehicle['m_s'] * Env['g'] * Vehicle['h_cog']

    Fzfl = Vehicle['M'] * Env['g'] * Vehicle['lr'] / Vehicle['L'] / 2 \
           - Vehicle['m_s']*ax*Vehicle['h_cog']/Vehicle['L']/2 \
           - Vehicle['m_s']*ax*Vehicle['h_cog']*Vehicle['lf']/Vehicle['L']**2 * (1 + Vehicle['K_sf']/Vehicle['K_sr']) / 2 \
           - F_Liftf/2 \
           - Vehicle['m_uf'] * Vehicle['h_uf'] / Vehicle['Bf'] * ay \
           - Vehicle['m_s'] * ay / Vehicle['Bf'] * (Vehicle['lr']/Vehicle['L'] * Vehicle['h_f'] + Vehicle['K_phif']*Vehicle['h_cog']/den)

    Fzfr = Fzfl + 2 * Vehicle['m_uf'] * Vehicle['h_uf'] / Vehicle['Bf'] * ay + 2 * Vehicle['m_s'] * ay / Vehicle['Bf'] * (Vehicle['lr']/Vehicle['L'] * Vehicle['h_f'] + Vehicle['K_phif']*Vehicle['h_cog']/den)

    Fzrl = Vehicle['M'] * Env['g'] * Vehicle['lf'] / Vehicle['L'] / 2 \
           + Vehicle['m_s']*ax*Vehicle['h_cog']/Vehicle['L']/2 \
           + Vehicle['m_s']*ax*Vehicle['h_cog']*Vehicle['lf']/Vehicle['L']**2 * (1 + Vehicle['K_sf']/Vehicle['K_sr']) / 2 \
           - F_Liftr/2 \
           - Vehicle['m_ur'] * Vehicle['h_ur'] / Vehicle['Br'] * ay \
           - Vehicle['m_s'] * ay / Vehicle['Br'] * (Vehicle['lf']/Vehicle['L'] * Vehicle['h_r'] + Vehicle['K_phir']*Vehicle['h_cog']/den)

    Fzrr = Fzrl + 2 * Vehicle['m_ur'] * Vehicle['h_ur'] / Vehicle['Br'] * ay + 2 * Vehicle['m_s'] * ay / Vehicle['Br'] * (Vehicle['lf']/Vehicle['L'] * Vehicle['h_r'] + Vehicle['K_phir']*Vehicle['h_cog']/den)

    # === 6. Tire Forces ===
    _, _, Fxfl, Fyfl, _ = tire_fun(Fzfl, Vehicle['Camber'], Sxfl, Safl)
    _, _, Fxfr, Fyfr, _ = tire_fun(Fzfr, Vehicle['Camber'], Sxfr, Safr)
    _, _, Fxrl, Fyrl, _ = tire_fun(Fzrl, Vehicle['Camber'], Sxrl, Sarl)
    _, _, Fxrr, Fyrr, _ = tire_fun(Fzrr, Vehicle['Camber'], Sxrr, Sarr)

    Fxfl *= Env['mu']
    Fxfr *= Env['mu']
    Fxrl *= Env['mu']
    Fxrr *= Env['mu']

    # === 7. Forces & Moment Sum ===
    Fx = Fxfl*cos(delta_fl) - Fyfl*sin(delta_fl) \
       + Fxfr*cos(delta_fr) - Fyfr*sin(delta_fr) \
       + Fxrl + Fxrr + F_drag

    Fy = Fyfl*cos(delta_fl) + Fxfl*sin(delta_fl) \
       + Fyfr*cos(delta_fr) + Fxfr*sin(delta_fr) \
       + Fyrl + Fyrr

    Mz = -(Fxfl*cos(delta_fl) - Fyfl*sin(delta_fl)) * Vehicle['Bf']/2 \
         + (Fxfr*cos(delta_fr) - Fyfr*sin(delta_fr)) * Vehicle['Bf']/2 \
         - Fxrl * Vehicle['Br']/2 + Fxrr * Vehicle['Br']/2 \
         + (Fyfl*cos(delta_fl) + Fxfl*sin(delta_fl)) * Vehicle['lf'] \
         + (Fyfr*cos(delta_fr) + Fxfr*sin(delta_fr)) * Vehicle['lf'] \
         - Fyrl * Vehicle['lr'] - Fyrr * Vehicle['lr']

    opti.subject_to(ax == Fx / Vehicle['M'])
    opti.subject_to(ay == Fy / Vehicle['M'])

    # === 8. Power Approximation ===
    def wheel_speed(v, dpsi, b, sx):
        v_wheel = Vel * cos(beta) + dpsi * b
        return if_else(sx >= 0, v_wheel / (1 - sx), v_wheel * (1 + sx)) / Vehicle['RollingRadius'] / 2 / pi * 60 * Vehicle['GearRatio']
    
    n_fl = fmin(wheel_speed(-Vehicle['Bf']/2, dpsi, -1, Sxfl), 20000)
    n_fr = fmin(wheel_speed(+Vehicle['Bf']/2, dpsi, 1, Sxfr), 20000)
    n_rl = fmin(wheel_speed(-Vehicle['Br']/2, dpsi, -1, Sxrl), 20000)
    n_rr = fmin(wheel_speed(+Vehicle['Br']/2, dpsi, 1, Sxrr), 20000)
    Max_fl = Vehicle['Trqmap'](n_fl)
    Max_fr = Vehicle['Trqmap'](n_fr)
    Max_rl = Vehicle['Trqmap'](n_rl)
    Max_rr = Vehicle['Trqmap'](n_rr)

    Fxmax_fl = Max_fl * Vehicle['GearRatio'] / Vehicle['RollingRadius']
    Fxmax_fr = Max_fr * Vehicle['GearRatio'] / Vehicle['RollingRadius']
    Fxmax_rl = Max_rl * Vehicle['GearRatio'] / Vehicle['RollingRadius']
    Fxmax_rr = Max_rr * Vehicle['GearRatio'] / Vehicle['RollingRadius']

    Power = Fxfl * Vehicle['RollingRadius'] / Vehicle['GearRatio'] * n_fl / 9549.297 \
          + Fxfr * Vehicle['RollingRadius'] / Vehicle['GearRatio'] * n_fr / 9549.297 \
          + Fxrl * Vehicle['RollingRadius'] / Vehicle['GearRatio'] * n_rl / 9549.297 \
          + Fxrr * Vehicle['RollingRadius'] / Vehicle['GearRatio'] * n_rr / 9549.297

    return opti, {
        'delta': delta, 'beta': beta, 'dpsi': dpsi,
        'Sxfl': Sxfl, 'Sxfr': Sxfr, 'Sxrl': Sxrl, 'Sxrr': Sxrr,
        'ax': ax, 'ay': ay, 'Mz': Mz, 'Power': Power,
        'Fx': Fx, 'Fy': Fy,
        'Fzfl': Fzfl, 'Fzfr': Fzfr, 'Fzrl': Fzrl, 'Fzrr': Fzrr,
        'Fxfl': Fxfl, 'Fyfl': Fyfl,
        'Fxfr': Fxfr, 'Fyfr': Fyfr,
        'Fxrl': Fxrl, 'Fyrl': Fyrl,
        'Fxrr': Fxrr, 'Fyrr': Fyrr,
        'delta_fl': delta_fl, 'delta_fr': delta_fr,
        'Safl': Safl, 'Safr': Safr, 'Sarl': Sarl, 'Sarr': Sarr,
        'F_Liftf': F_Liftf, 'F_Liftr': F_Liftr, 'F_drag': F_drag,
        'n_fl': n_fl, 'n_fr': n_fr, 'n_rl': n_rl, 'n_rr': n_rr,
        'Max_fl': Max_fl, 'Max_fr': Max_fr, 'Max_rl': Max_rl, 'Max_rr': Max_rr,
        'Fxmax_fl': Fxmax_fl, 'Fxmax_fr': Fxmax_fr, 'Fxmax_rl': Fxmax_rl, 'Fxmax_rr': Fxmax_rr,
    }

# Define Vehicle and Environment Parameters

In [ ]:
# 速度矢量 km/h
velocities = np.arange(20, 121, 10)
ax_steps = 11

# 预分配
GGV_ax = np.zeros((len(velocities), ax_steps))
GGV_ay = np.zeros((len(velocities), ax_steps))
GGV_delta = np.zeros((len(velocities), ax_steps))
GGV_beta = np.zeros((len(velocities), ax_steps))
GGV_Power = np.zeros((len(velocities), ax_steps))

Vehicle = {}
Env = {}

# Sprung Mass
Vehicle['m_s'] = 240     # vehicle sprung mass (kg)
Vehicle['lf'] = 0.8      # distance COG to front axle (m)
Vehicle['lr'] = 0.8      # distance COG to rear axle (m)
Vehicle['L'] = Vehicle['lf'] + Vehicle['lr']
Vehicle['Bf'] = 1.2
Vehicle['Br'] = 1.2
Vehicle['h_cog'] = 0.5   # Sprung Mass COG height (m)

# Unsprung Mass
Vehicle['m_uf'] = 10     # vehicle front unsprung mass (kg)
Vehicle['m_ur'] = 10     # vehicle rear unsprung mass (kg)
Vehicle['h_uf'] = 0.25   # Front Unsprung Mass COG height (m)
Vehicle['h_ur'] = 0.25   # Rear Unsprung Mass COG height (m)

Vehicle['M'] = Vehicle['m_s'] + Vehicle['m_uf'] + Vehicle['m_ur']

# Suspension
Vehicle['K_sf'] = 21400  # Front Spring Stiffness (N/m)
Vehicle['K_sr'] = 24000  # Rear Spring Stiffness (N/m)
Vehicle['K_phif'] = 39525  # Front Axle Rolling Stiffness (N/rad)
Vehicle['K_phir'] = 47792  # Rear Axle Rolling Stiffness (N/rad)
Vehicle['h_f'] = 0.2     # Front Roll Center height (m)
Vehicle['h_r'] = 0.2     # Rear Roll Center height (m)

# Aero
Vehicle['ClA'] = -5.93
Vehicle['CdA'] = -2.05
Vehicle['CoP'] = 0.45
Vehicle['rho'] = 1.2258

# Powertrain
Vehicle['GearRatio'] = 12
Vehicle['RollingRadius'] = 0.195
Vehicle['ngrid'] = [0, 12000, 16000, 20000]
Vehicle['TrqMax'] = [21, 21, 12, 8]
Vehicle['Trqmap'] = interpolant('LUT', 'linear', [Vehicle['ngrid']], Vehicle['TrqMax'])
Vehicle['P_max'] = 80    # maximum Power (kW)

# Steering (转换角度为弧度)
Vehicle['SWA'] = np.array([-100, 0, 100]) * np.pi / 180
Vehicle['delta_fl'] = np.array([-33, 0, 33]) * np.pi / 180
Vehicle['delta_fr'] = np.array([-33, 0, 33]) * np.pi / 180

Vehicle['Strmap_fl'] = ca.interpolant('Strmap_fl', 'linear', [Vehicle['SWA']], Vehicle['delta_fl'])
Vehicle['Strmap_fr'] = ca.interpolant('Strmap_fr', 'linear', [Vehicle['SWA']], Vehicle['delta_fr'])

# Brake
Vehicle['Max_Mech_FrontBrake'] = 3000  # N
Vehicle['Max_Mech_RearBrake'] = 2000   # N

# Tuning
Vehicle['Camber'] = 0
Vehicle['Static_Toe_Front'] = 0.1*pi/180 # rad !内八为正!
Vehicle['Static_Toe_Rear'] = 0.1*pi/180 # rad

# Environment
Env['mu'] = 1.0
Env['g'] = 9.81

# GGV Optimization

In [ ]:
tire = tire_Li
for idx, vel_kmh in enumerate(velocities):
    Vel = vel_kmh / 3.6 + 1e-5  # 转为 m/s，加个小eps避免除零

    # 初始化结构体GGS
    GGS = {
        'ax': np.zeros(ax_steps),
        'ay': np.zeros(ax_steps),
        'delta': np.zeros(ax_steps),
        'beta': np.zeros(ax_steps),
        'Power': np.zeros(ax_steps),
        'axacc': np.zeros((ax_steps - 1) // 2 + 1),
        'ayacc': np.zeros((ax_steps - 1) // 2 + 1),
        'deltaacc': np.zeros((ax_steps - 1) // 2 + 1),
        'betaacc': np.zeros((ax_steps - 1) // 2 + 1),
        'Poweracc': np.zeros((ax_steps - 1) // 2 + 1),
        'axbrk': np.zeros((ax_steps - 1) // 2 + 1),
        'aybrk': np.zeros((ax_steps - 1) // 2 + 1),
        'deltabrk': np.zeros((ax_steps - 1) // 2 + 1),
        'betabrk': np.zeros((ax_steps - 1) // 2 + 1),
        'Powerbrk': np.zeros((ax_steps - 1) // 2 + 1),
    }

    ####### 最大纵向加速度求解 #######
    opti, v = build_vehicle_model_opti(Vel, Vehicle, Env, tire)

    # 目标函数，最大化ax，CasADi里最小化负的ax
    opti.minimize(-v['ax'])

    # 变量初值
    opti.set_initial(v['delta'], 0)
    opti.set_initial(v['beta'], 0)
    opti.set_initial(v['dpsi'], 0)
    opti.set_initial(v['Sxfl'], 0)
    opti.set_initial(v['Sxfr'], 0)
    opti.set_initial(v['Sxrl'], 0)
    opti.set_initial(v['Sxrr'], 0)

    # 约束
    opti.subject_to(v['Power'] <= 80)  # 功率约束
    opti.subject_to(opti.bounded(-1e-3, v['ay'], 1e-3))
    opti.subject_to(opti.bounded(-1e-3, v['dpsi'], 1e-3))
    opti.subject_to(opti.bounded(-1e-3, v['beta'], 1e-3))
    opti.subject_to(opti.bounded(-1e-3, v['delta'], 1e-3))

    opti.subject_to(v['Sxfl'] >= 0)
    opti.subject_to(v['Sxfr'] >= 0)
    opti.subject_to(v['Sxrl'] >= 0)
    opti.subject_to(v['Sxrr'] >= 0)
    opti.subject_to(v['Sxfl'] == v['Sxfr'])
    opti.subject_to(v['Sxrl'] == v['Sxrr'])

    opti.subject_to(v['Fzfl'] >= 0)
    opti.subject_to(v['Fzfr'] >= 0)
    opti.subject_to(v['Fzrl'] >= 0)
    opti.subject_to(v['Fzrr'] >= 0)
    opti.subject_to(v['Fzfl'] == v['Fzfr'])
    opti.subject_to(v['Fzrl'] == v['Fzrr'])

    opti.subject_to(v['Fxfl'] <= v['Fxmax_fl'])
    opti.subject_to(v['Fxfr'] <= v['Fxmax_fr'])
    opti.subject_to(v['Fxrl'] <= v['Fxmax_rl'])
    opti.subject_to(v['Fxrr'] <= v['Fxmax_rr'])

    # 求解设置
    opts = {
            'ipopt': {
                'print_level': 0,
                'max_iter': 8000,
                'tol': 1e-2,
                'nlp_scaling_method': 'gradient-based',  # IPOPT自动缩放方法之一
            }
        }
    opti.solver('ipopt', opts)
    sol = opti.solve()

    # 提取结果
    GGS['ax'][0] = sol.value(v['ax'])
    GGS['ay'][0] = sol.value(v['ay'])
    GGS['delta'][0] = sol.value(v['delta'])
    GGS['beta'][0] = sol.value(v['beta'])
    GGS['Power'][0] = sol.value(v['Power'])

    ####### 最大制动减速度求解 #######
    opti, v = build_vehicle_model_opti(Vel, Vehicle, Env, tire)

    opti.minimize(v['ax'])

    opti.set_initial(v['delta'], 0)
    opti.set_initial(v['beta'], 0)
    opti.set_initial(v['dpsi'], 0)
    opti.set_initial(v['Sxfl'], 0)
    opti.set_initial(v['Sxfr'], 0)
    opti.set_initial(v['Sxrl'], 0)
    opti.set_initial(v['Sxrr'], 0)

    opti.subject_to(opti.bounded(-1e-3, v['ay'], 1e-3))
    opti.subject_to(opti.bounded(-1e-3, v['dpsi'], 1e-3))
    opti.subject_to(opti.bounded(-1e-3, v['beta'], 1e-3))
    opti.subject_to(opti.bounded(-1e-3, v['delta'], 1e-3))

    opti.subject_to(v['Sxfl'] <= 0)
    opti.subject_to(v['Sxfr'] <= 0)
    opti.subject_to(v['Sxrl'] <= 0)
    opti.subject_to(v['Sxrr'] <= 0)
    opti.subject_to(v['Sxfl'] == v['Sxfr'])
    opti.subject_to(v['Sxrl'] == v['Sxrr'])

    opti.subject_to(v['Fzfl'] >= 0)
    opti.subject_to(v['Fzfr'] >= 0)
    opti.subject_to(v['Fzrl'] >= 0)
    opti.subject_to(v['Fzrr'] >= 0)
    opti.subject_to(v['Fzfl'] == v['Fzfr'])
    opti.subject_to(v['Fzrl'] == v['Fzrr'])

    opti.subject_to(v['Fxfl'] >= -2000)
    opti.subject_to(v['Fxfr'] >= -2000)
    opti.subject_to(v['Fxrl'] >= -1000)
    opti.subject_to(v['Fxrr'] >= -1000)

    opti.solver('ipopt', opts)
    sol = opti.solve()

    GGS['ax'][-1] = sol.value(v['ax'])
    GGS['ay'][-1] = sol.value(v['ay'])
    GGS['delta'][-1] = sol.value(v['delta'])
    GGS['beta'][-1] = sol.value(v['beta'])
    GGS['Power'][-1] = sol.value(v['Power'])

    ####### 最大横向加速度求解 #######
    GGS['axacc'] = np.linspace(GGS['ax'][0], 0, (ax_steps - 1)//2 + 1)

    for j in range(1, len(GGS['axacc'])):
        opti, v = build_vehicle_model_opti(Vel, Vehicle, Env, tire)

        # 这里k=1，SxFactor=1e5，权衡目标函数
        k = 1.0
        SxFactor = 1e2
        opti.minimize(-(k * v['ay'] + (1 - k) * v['Mz'])/10 + SxFactor * (
            v['Sxfl'] ** 2 + v['Sxfr'] ** 2 + v['Sxrl'] ** 2 + v['Sxrr'] ** 2))

        opti.subject_to(v['ay'] - Vel * v['dpsi'] == 0)
        opti.subject_to(v['Mz'] == 0)
        opti.subject_to(v['ax'] == GGS['axacc'][j])

        opti.subject_to(v['Fzfl'] >= 0)
        opti.subject_to(v['Fzfr'] >= 0)
        opti.subject_to(v['Fzrl'] >= 0)
        opti.subject_to(v['Fzrr'] >= 0)
        opti.subject_to(v['Fxfl'] <= v['Fxmax_fl'])
        opti.subject_to(v['Fxfr'] <= v['Fxmax_fr'])
        opti.subject_to(v['Fxrl'] <= v['Fxmax_rl'])
        opti.subject_to(v['Fxrr'] <= v['Fxmax_rr'])
        opti.subject_to(v['Power'] <= 80)  # 功率约束

        opti.set_initial(v['delta'], np.deg2rad(0))
        opti.set_initial(v['beta'], np.deg2rad(0))
        opti.set_initial(v['dpsi'], 0)

        opts = {
            'ipopt': {
                'print_level': 0,
                'max_iter': 8000,
                'tol': 1e-2,
                'nlp_scaling_method': 'gradient-based',  # IPOPT自动缩放方法之一
            }
        }
        opti.solver('ipopt', opts)
        sol = opti.solve()

        GGS['ayacc'][j] = sol.value(v['ay'])
        GGS['deltaacc'][j] = sol.value(v['delta'])
        GGS['betaacc'][j] = sol.value(v['beta'])
        GGS['Poweracc'][j] = sol.value(v['Power'])

    # 把acc部分赋值回去GGS主数组
    GGS['ax'][1:(ax_steps+1)//2] = GGS['axacc'][1:]
    GGS['ay'][1:(ax_steps+1)//2] = GGS['ayacc'][1:]
    GGS['delta'][1:(ax_steps+1)//2] = GGS['deltaacc'][1:]
    GGS['beta'][1:(ax_steps+1)//2] = GGS['betaacc'][1:]
    GGS['Power'][1:(ax_steps+1)//2] = GGS['Poweracc'][1:]

    GGS['axbrk'] = np.linspace(0, GGS['ax'][-1], (len(GGS['ax']) - 1)//2 + 1)

    for i_ in range(1, len(GGS['axbrk'])):
        opti, v = build_vehicle_model_opti(Vel, Vehicle, Env, tire)
    
        k = 1.0
        SxFactor = 1e2
    
        obj = -(k * v['ay'] + (1 - k) * v['Mz']) / 10 + SxFactor * (
            v['Sxfl']**2 + v['Sxfr']**2 + v['Sxrl']**2 + v['Sxrr']**2
        )
        opti.minimize(obj)
    
        opti.subject_to(v['ay'] - Vel * v['dpsi'] == 0)
        opti.subject_to(v['Mz'] == 0)
        opti.subject_to(v['ax'] == GGS['axbrk'][i_])
    
        opti.subject_to(v['Fzfl'] >= 0)
        opti.subject_to(v['Fzfr'] >= 0)
        opti.subject_to(v['Fzrl'] >= 0)
        opti.subject_to(v['Fzrr'] >= 0)
        opti.subject_to(v['Fxfl'] >= -2000)
        opti.subject_to(v['Fxfr'] >= -2000)
        opti.subject_to(v['Fxrl'] >= -1000)
        opti.subject_to(v['Fxrr'] >= -1000)
    
        opti.set_initial(v['delta'], np.deg2rad(0))
        opti.set_initial(v['beta'], np.deg2rad(0))
        opti.set_initial(v['dpsi'], 0)

        opti.solver('ipopt', opts)
        sol = opti.solve()
    
        GGS['aybrk'][i_] = sol.value(v['ay'])
        GGS['deltabrk'][i_] = sol.value(v['delta'])
        GGS['betabrk'][i_] = sol.value(v['beta'])
        GGS['Powerbrk'][i_] = sol.value(v['Power'])
    
    # 赋值回主数组
    start_idx = (ax_steps + 1)//2
    end_idx = len(GGS['ax']) - 1
    
    GGS['ax'][start_idx:end_idx] = GGS['axbrk'][1:-1]
    GGS['ay'][start_idx:end_idx] = GGS['aybrk'][1:-1]
    GGS['delta'][start_idx:end_idx] = GGS['deltabrk'][1:-1]
    GGS['beta'][start_idx:end_idx] = GGS['betabrk'][1:-1]
    GGS['Power'][start_idx:end_idx] = GGS['Powerbrk'][1:-1]

    # 保存结果到GGV
    GGV_ax[idx, :] = GGS['ax']
    GGV_ay[idx, :] = GGS['ay']
    GGV_delta[idx, :] = GGS['delta']
    GGV_beta[idx, :] = GGS['beta']
    GGV_Power[idx, :] = GGS['Power']

# GGV Visualization

In [ ]:
# 绘制GGV图
plt.figure(figsize=(10,8))
for i in range(len(velocities)):
    plt.plot(GGV_ay[i, :], GGV_ax[i, :], '-d', label=f'{velocities[i]} km/h')
plt.xlabel('a_y (m/s²)')
plt.ylabel('a_x (m/s²)')
plt.title('GGV Diagram')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
GGV_ax_full = np.concatenate([np.flip(GGV_ax, axis=1), GGV_ax], axis=1)  # [num_speeds, 2*num_points]
GGV_ay_full = np.concatenate([-np.flip(GGV_ay, axis=1), GGV_ay], axis=1)

# 转换为 g 单位
GGV_ax_g = GGV_ax_full / 9.81
GGV_ay_g = GGV_ay_full / 9.81

Z_speed = np.array(velocities)[:, np.newaxis]  # [num_speeds, 1]
GGV_speed = np.tile(Z_speed, (1, GGV_ax_full.shape[1]))  # [num_speeds, 2*num_points]

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(
    GGV_ay_g,       # X: 横向加速度
    GGV_ax_g,       # Y: 纵向加速度
    GGV_speed,      # Z: 车速
    cmap='viridis',
    edgecolor='k',
    alpha=0.8
)

# 坐标轴和标题
ax.set_xlabel('Lateral Acceleration (g)')
ax.set_ylabel('Longitudinal Acceleration (g)')
ax.set_zlabel('Vehicle Speed (km/h)')
ax.set_title('3D GGV Surface (Z-axis: Speed)')
fig.colorbar(surf, ax=ax, label='Speed (km/h)')

plt.tight_layout()
plt.show()

# QSS Laptime Simulation based on GGV

In [ ]:
# 清零很小的ay值
GGV_ay[np.abs(GGV_ay) < 1e-5] = 0

m, n = GGV_ax.shape

# 初始化临时数组
accmap_temp = np.zeros((2, n, m))
brkmap_temp = np.zeros((2, n, m))

for i in range(m):
    # 加速映射，只选ax >= 0
    for j in range(n):
        if GGV_ax[i, j] >= 0:
            accmap_temp[0, j, i] = GGV_ay[i, j]
            accmap_temp[1, j, i] = GGV_ax[i, j]
    # 制动映射，只选ax <= 0
    count = 0
    for j in range(n):
        if GGV_ax[i, j] <= 0:
            brkmap_temp[0, count, i] = GGV_ay[i, j]
            brkmap_temp[1, count, i] = GGV_ax[i, j]
            count += 1

# 标准重力加速度换算
accmap = accmap_temp / 9.81
brkmap = brkmap_temp / 9.81
pureay_map = np.max(GGV_ay, axis=1) / 9.81

In [ ]:
# ---------- 2. 读取赛道数据与路径插值 ----------
track_data = loadmat('Trackdata20.mat')
X = track_data['X'].flatten()
Y = track_data['Y'].flatten()
VehicleSpeed = track_data['VehicleSpeed'].flatten()

sections = 1200
t = np.arange(1, len(X) + 1)
x_new = np.linspace(1, t[-1], sections)

ppv_x = PchipInterpolator(t, X)
ppv_y = PchipInterpolator(t, Y)

vehicle_path_x = ppv_x(x_new)
vehicle_path_y = ppv_y(x_new)

# ---------- 3. 计算轨迹长度与曲率 ----------
def arclength(x, y):
    dx = np.diff(x)
    dy = np.diff(y)
    return np.sum(np.sqrt(dx ** 2 + dy ** 2))

def curvature(x, y):
    dx = np.gradient(x)
    dy = np.gradient(y)
    ddx = np.gradient(dx)
    ddy = np.gradient(dy)
    numerator = dx * ddy - dy * ddx
    denominator = (dx**2 + dy**2)**1.5
    with np.errstate(divide='ignore', invalid='ignore'):
        k = np.abs(numerator) / denominator
        k[np.isnan(k)] = 0
    return k

path_length = arclength(vehicle_path_x, vehicle_path_y)
KT = curvature(vehicle_path_x, vehicle_path_y)

# 转弯半径，防止过小设阈值8m
RT = np.zeros_like(KT)
RT[KT > 0] = 1.0 / KT[KT > 0]
RT[RT < 8] = 8
RT[RT == 0] = 8  # 曲率为0时也设8

In [ ]:
# ---------- 4. 速度极限及角点计算 ----------
def find_maxacc(velosityrange, velosity, accmap, ay, pureay_map):
    index = np.searchsorted(velosityrange, velosity) - 1
    index = np.clip(index, 0, len(velosityrange)-2)
    max_ax1 = interp1d(accmap[0, :, index], accmap[1, :, index], fill_value='extrapolate')(ay)
    max_ax1 = max(0, max_ax1)
    max_ax2 = interp1d(accmap[0, :, index+1], accmap[1, :, index+1], fill_value='extrapolate')(ay)
    max_ax2 = max(0, max_ax2)
    max_ax = np.interp(velosity, [velosityrange[index], velosityrange[index+1]], [max_ax1, max_ax2])
    return max_ax, max_ax1, max_ax2, index

def find_minacc(velosityrange, velosity, brkmap, ay, pureay_map):
    index = np.searchsorted(velosityrange, velosity) - 1
    index = np.clip(index, 0, len(velosityrange)-2)
    min_ax1 = interp1d(brkmap[0, :, index], brkmap[1, :, index], fill_value='extrapolate')(ay)
    min_ax1 = min(min_ax1, 0)
    min_ax2 = interp1d(brkmap[0, :, index+1], brkmap[1, :, index+1], fill_value='extrapolate')(ay)
    min_ax2 = min(min_ax2, 0)
    min_ax = np.interp(velosity, [velosityrange[index], velosityrange[index+1]], [min_ax1, min_ax2])
    return min_ax, min_ax1, min_ax2, index

def find_cornering(velosityrange, RT, pureay_map):
    V_cornering = np.zeros(len(RT))
    LatA_cornering = np.zeros(len(RT))
    for j in range(len(RT)):
        if j % 10 == 0:
            print(j, 'sections done')
        for i in np.arange(velosityrange[0], velosityrange[-1], 0.1):
            V_temp = i / 3.6
            ay_t = V_temp**2 / RT[j]
            ay_v = interp1d(velosityrange, pureay_map, fill_value='extrapolate')(i) * 9.81
            if ay_t > ay_v:
                V_cornering[j] = i - 1
                LatA_cornering[j] = ((i - 1) / 3.6)**2 / RT[j]
                break
            if i >= velosityrange[-1] - 0.1:
                V_cornering[j] = velosityrange[-1]
                LatA_cornering[j] = ((i - 1) / 3.6)**2 / RT[j]
    Apex_index = []
    for i in range(1, len(V_cornering) - 1):
        if V_cornering[i-1] > V_cornering[i] <= V_cornering[i+1]:
            Apex_index.append(i)
    return V_cornering, LatA_cornering, Apex_index

# ---------- 5. 计算加速与制动速度曲线 ----------
[V_cornering, LatA_cornering, Apex_index] = find_cornering(velocities, RT, pureay_map)
Apex_index = np.concatenate(([0], Apex_index, [sections-1]))

In [ ]:
# 初始化速度数组
V_init =35
V_accf = np.zeros(sections)
V_accf[0] = V_init

acc_xf = np.zeros(sections)
tt_accf = np.zeros(sections)
dist = np.zeros(sections)
for i in range(len(Apex_index) - 1):
    for j in range(Apex_index[i], Apex_index[i+1]):
        dist[j] = np.sqrt((vehicle_path_x[j] - vehicle_path_x[j+1])**2+(vehicle_path_y[j] - vehicle_path_y[j+1])**2)
        acc_xf[j], _, _, _ = find_maxacc(velocities, V_accf[j], accmap, min((V_accf[j]/3.6)**2/RT[j]/9.81, LatA_cornering[j]/9.81), pureay_map)
        # 计算时间tt_accf[j]，求解 0.5*a*t^2 + v*t - dist = 0
        a = 0.5 * acc_xf[j] * 9.81
        b = V_accf[j] / 3.6
        c = -dist[j]
        roots = np.roots([a, b, c])
        roots = roots[np.isreal(roots)]
        tt_accf[j] = np.max(np.real(roots)) if len(roots) > 0 else 0
        vmax = min(velocities[-1], V_cornering[j+1])
        V_accf[j+1] = min((V_accf[j]/3.6 + acc_xf[j]*9.81*tt_accf[j]) * 3.6, vmax)
        vmin = velocities[0]
        V_accf[j+1] = max(V_accf[j+1], vmin)
# plt.plot(V_accf)
# 计算制动速度曲线
V_init = V_accf[-1]
V_brkb = np.zeros(sections)
V_brkb[-1] = V_init

brk_xb = np.zeros(sections)
tt_brkb = np.zeros(sections)
for i in range(len(Apex_index)-1, 0, -1):
    for j in range(Apex_index[i], Apex_index[i-1], -1):
        dist[j] = np.hypot(vehicle_path_x[j] - vehicle_path_x[j-1], vehicle_path_y[j] - vehicle_path_y[j-1])
        brk_xb[j], _, _, _ = find_minacc(velocities, V_brkb[j], brkmap, min((V_brkb[j]/3.6)**2 / RT[j] / 9.81, LatA_cornering[j] / 9.81), pureay_map)
        brk_xb[j] = -brk_xb[j]
        a = 0.5 * brk_xb[j] * 9.81
        b = V_brkb[j] / 3.6
        c = -dist[j]
        roots = np.roots([a, b, c])
        roots = roots[np.isreal(roots)]
        tt_brkb[j] = np.max(np.real(roots)) if len(roots) > 0 else 0
        vmax = min(velocities[-1], V_cornering[j-1])
        V_brkb[j-1] = min((V_brkb[j]/3.6 + brk_xb[j]*9.81*tt_brkb[j]) * 3.6, vmax)
        V_brkb[j-1] = max(V_brkb[j-1], velocities[0])

# plt.plot(V_brkb)
######### TODO: ADD HOTLAP #########

# Laptime Visualization

In [ ]:
# ---------- 6. 综合加速和制动速度，计算总速度 ----------
V = np.minimum(V_accf, V_brkb)

# ---------- 7. 计算路径长度与时间 ----------
disttotal = np.zeros(sections)
t = np.zeros(sections)
tt = np.zeros(sections - 1)

for i in range(sections - 1):
    dist[i] = np.hypot(vehicle_path_x[i] - vehicle_path_x[i+1], vehicle_path_y[i] - vehicle_path_y[i+1])
    disttotal[i+1] = disttotal[i] + dist[i]
    avg_speed = (V[i] + V[i+1]) / 2 / 3.6  # m/s
    if avg_speed > 0:
        tt[i] = dist[i] / avg_speed
    else:
        tt[i] = 0
    t[i+1] = t[i] + tt[i]

# ---------- 8. 绘图 ----------
plt.figure()
plt.plot(disttotal, V_accf, label='Accelerate', linewidth=1)
plt.plot(disttotal, V_brkb, label='Brake', linewidth=1)
plt.plot(disttotal, V_cornering, label='Cornering', linestyle='--', linewidth=1)
plt.xlabel('Distance (m)')
plt.ylabel('Vehicle Speed (kph)')
plt.legend()
plt.grid()
plt.show()

In [ ]:
dist_act = np.zeros(len(X))
for i in range(len(X)-1):
    dist_act[i+1] = dist_act[i] + np.hypot(X[i] - X[i+1], Y[i] - Y[i+1])
# ---------- 9. 参考实际车辆速度比较（假设VehicleSpeed变量可用） ----------
plt.figure()
plt.plot(disttotal, V, label='Reference')
plt.plot(dist_act, VehicleSpeed * 3.6, label='Actual')
plt.legend()
plt.xlabel('Distance (m)')
plt.ylabel('Vehicle Speed (kph)')
plt.grid()
plt.show()